In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Dropout, Flatten

# Neural Networks for Images

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [4]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [5]:
X_train = tf.expand_dims(X_train, -1).numpy()
X_test = tf.expand_dims(X_test, -1).numpy()

In [6]:
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES = len(set(y_train))

In [7]:
INPUT_SHAPE, NUM_CLASSES

((28, 28, 1), 10)

In [18]:
tf.keras.backend.clear_session()

In [19]:
model = Sequential([
    Input(INPUT_SHAPE),
    Conv2D(32, 3, padding = 'same', strides = 2, activation = 'relu'),
    Conv2D(32, 3, padding = 'same', activation = 'relu'),
    MaxPool2D(),
    Conv2D(16, 3, padding = 'same', strides = 2, activation = 'relu'),
    Conv2D(16, 3, padding = 'same', activation = 'relu'),

    Flatten(),

    Dense(NUM_CLASSES, activation = 'relu'),
    Dropout(0.05),
    Dense(NUM_CLASSES, activation = 'softmax')
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 7, 7, 32)          0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 16)          4624      
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 16)          2320      
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                        

In [21]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [22]:
history = model.fit(X_train, y_train, epochs = 20, validation_split = 0.1)

Epoch 1/20
1688/1688 [==============================] - 11s 5ms/step - loss: 0.4962 - accuracy: 0.8341 - val_loss: 0.1091 - val_accuracy: 0.9720
Epoch 2/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1933 - accuracy: 0.9369 - val_loss: 0.0800 - val_accuracy: 0.9787
Epoch 3/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1386 - accuracy: 0.9589 - val_loss: 0.0713 - val_accuracy: 0.9825
Epoch 4/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1036 - accuracy: 0.9706 - val_loss: 0.0735 - val_accuracy: 0.9793
Epoch 5/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0794 - accuracy: 0.9767 - val_loss: 0.0528 - val_accuracy: 0.9847
Epoch 6/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0654 - accuracy: 0.9805 - val_loss: 0.0743 - val_accuracy: 0.9785
Epoch 7/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0592 - accuracy: 0.9821 - val_loss: 0.0535 - val_accuracy

KeyboardInterrupt: 

In [23]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0738 - accuracy: 0.9837


[0.07376530766487122, 0.9836999773979187]

In [24]:
model.save_weights('model.h5')